<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# 필요 라이브러리 설치
!pip install PyPDF2 chromadb sentence-transformers pandas

ERROR: Could not find a version that satisfies the requirement pyhwp2txt (from versions: none)
ERROR: No matching distribution found for pyhwp2txt


In [25]:
# HWP -> PDF 변환 코드 : 독립적으로 먼저 파일 변환
# 리눅스 환경에서 HWP 파일을 PDF 변환이 안됨
# 별거 다 해봤는데 답 없다... 한땀한땀 pdf로 손으로 변경하자

In [28]:
# 구글 드라이브 document_files에 있는 파일들 전부 저장
import os
import re
import unicodedata
import pytz
import pandas as pd
from datetime import datetime
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from chromadb import PersistentClient
import subprocess   # hwp -> pdf 변환

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
ROOT_FOLDER = "/content/drive/MyDrive/test_files"     # 여기만 바꾸기(경로)
URL_MAPPING_SUFFIX = "_url.xlsx"

KST = pytz.timezone('Asia/Seoul')
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)
    return text.strip()

def split_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

def read_file(filepath):
    if filepath.endswith('.pdf'):
        reader = PdfReader(filepath)
        return "".join([p.extract_text() for p in reader.pages if p.extract_text()])
    elif filepath.endswith('.md'):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

def get_all_files(folder):
    all_files = []
    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            if f.endswith(('.pdf', '.md')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

def extract_category(file_path, base_folder):
    rel_path = os.path.relpath(file_path, start=base_folder)
    parts = rel_path.split(os.sep)
    return parts[0] if len(parts) > 1 else "기타"

def build_metadata(file_path, base_folder, base_name, source_url, ext):
    category = extract_category(file_path, base_folder)
    department_path = os.path.relpath(base_folder, ROOT_FOLDER)

    return {
        "file_name": base_name,
        "department": department_path,
        "category": category,
        "source_type": "pdf" if ext == ".pdf" else "markdown",
        "source_url": source_url,
        "date": datetime.now(KST).isoformat()
    }

total_added = 0

# 엑셀 기준으로 저장
for dirpath, _, filenames in os.walk(ROOT_FOLDER):
    for filename in filenames:
        if filename.endswith(URL_MAPPING_SUFFIX):
            folder = dirpath  # 엑셀 위치 폴더 = 기준 폴더
            dept_folder_name = os.path.basename(folder)
            url_file = os.path.join(folder, filename)

            try:
                url_df = pd.read_excel(url_file)
                url_mapping = dict(zip(url_df['파일명'], url_df['URL']))
            except Exception as e:
                print(f"❌ URL 매핑 로드 실패: {url_file} ({e})")
                continue

            file_paths = get_all_files(folder)

            for file_path in file_paths:
                file_name = os.path.basename(file_path)
                base_name = unicodedata.normalize("NFC", os.path.splitext(file_name)[0])
                ext = os.path.splitext(file_name)[1].lower()

                if file_name.endswith(URL_MAPPING_SUFFIX):
                    continue

                raw_text = read_file(file_path)
                if not raw_text.strip():
                    print(f"⚠️ {file_name}: 텍스트 없음 → 스킵")
                    continue

                cleaned = clean_text(raw_text)
                chunks = split_chunks(cleaned)
                embeddings = model.encode(chunks).tolist()
                ids = [f"{dept_folder_name}_{base_name}_chunk_{i}" for i in range(len(chunks))]

                new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

                for chunk, emb, id_ in zip(chunks, embeddings, ids):
                    if id_ not in existing_ids:
                        meta = build_metadata(file_path, folder, base_name, url_mapping.get(base_name, "출처 URL 미등록"), ext)
                        new_chunks.append(chunk)
                        new_embeddings.append(emb)
                        new_ids.append(id_)
                        new_metadatas.append(meta)

                if new_chunks:
                    collection.add(
                        documents=new_chunks,
                        embeddings=new_embeddings,
                        metadatas=new_metadatas,
                        ids=new_ids
                    )
                    print(f"✅ {file_name}: {len(new_chunks)}개 저장 완료")
                    total_added += len(new_chunks)
                else:
                    print(f"⚠️ {file_name}: 중복 청크만 존재 → 스킵")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")

✅ 학과소개.md: 2개 저장 완료
✅ 비전.md: 2개 저장 완료
✅ 연혁.md: 3개 저장 완료
✅ 인사말.md: 2개 저장 완료
✅ 소프트웨어학과.md: 2개 저장 완료
✅ 대학원.md: 1개 저장 완료
✅ 찾아오시는길.md: 1개 저장 완료
✅ 교수진.md: 3개 저장 완료
✅ SW중심대학사업 2025학년도 1학기 산학캡스톤디자인 모집공고.md: 3개 저장 완료
✅ 2024학년도 동계 계절학기 미래설계(전필) 1~6 증빙 파일 제출 안내.md: 3개 저장 완료
✅ K-Value up 미래설계 증빙파일 제출 안내.md: 3개 저장 완료
✅ 디지털기업-청년인재 간의 만남의 장 신청 안내.md: 3개 저장 완료
✅ 2025학년도 1학기 성적우수장학금 증빙 확인 바람_0212.md: 4개 저장 완료
✅ 자기주도적 역량강화 프로그램 장학금 신청 안내.md: 2개 저장 완료
✅ [전학년] 2025학년도 1학기 수강신청 안내.md: 2개 저장 완료
✅ 2025학년도 신입생 OT자료_과목수정.md: 1개 저장 완료
✅ 2025 신입생 안내교육(OT) 자료.pdf: 7개 저장 완료
✅ 2025학년도 1학기 추가수강 안내.md: 3개 저장 완료
✅ 2024학년도 취업역량강화 장학생 선발 안내.md: 3개 저장 완료
✅ 학생회 앨범.md: 1개 저장 완료
✅ 소프트웨어하

In [31]:
from chromadb import PersistentClient

# 설정
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 전체 문서 불러오기
results = collection.get(limit=None, include=["documents", "metadatas", "embeddings"])

print(f"\n📦 전체 저장된 벡터 수: {len(results['ids'])}")
print("=" * 70)

for i in range(len(results["ids"])):
    doc_id = results["ids"][i]
    metadata = results["metadatas"][i]
    doc_text = results["documents"][i].strip().replace("\n", " ")
    embedding = results["embeddings"][i]

    print(f"🔹 ID: {doc_id}")
    print(f"   📁 파일명: {metadata.get('file_name')}")
    print(f"   🏫 부서: {metadata.get('department')}")
    print(f"   📂 카테고리: {metadata.get('category')}")
    print(f"   📄 타입: {metadata.get('source_type')}")
    print(f"   🌐 URL: {metadata.get('source_url')}")
    print(f"   📅 날짜: {metadata.get('date')[:10]}")
    print(f"   💬 청크 문장: {doc_text}")
    print(f"   🧠 벡터 길이: {len(embedding)}")
    print(f"   🔢 벡터 앞 5개 값: {embedding[:5]}") # 384개 중에 앞 5개만 보여줌
    print("-" * 70)


📦 전체 저장된 벡터 수: 225
🔹 ID: Software Department (소프트웨어학과)_학과소개_chunk_0
   📁 파일명: 학과소개
   🏫 부서: department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)
   📂 카테고리: 학과소개
   📄 타입: markdown
   🌐 URL: https://sw.kongju.ac.kr/ZD1180/11621/subview.do
   📅 날짜: 2025-04-04
   💬 청크 문장: # 학과소개 **출처:** [https://sw.kongju.ac.kr/ZD1180/11621/subview.do](https://sw.kongju.ac.kr/ZD1180/11621/subview.do) ## 본문 Welcome to Department of Software 소프트웨어학과 홈페이지를 방문하신 네티즌 여러분 진심으로 환영합니다. 공주대학교 소프트웨어학과에서는 컴퓨터소프트웨어 인력양성에 필요한 다양하고 폭넓은 소프트웨어 교과목을 준비하였습니다. 인공지능을 비롯한 사물인터넷, ERP, 모바일, 웹, 게임, 서버 관리에 이르기까지 수많은 응용 분야에 진출할 수 있도록 이론뿐만 아니라 실습에 많은 시간을 할애하였습니다. 입학 후에는 수많은 컴퓨터프로그래밍 언어와 씨름하면서 코딩 능력을 향상시키도록 하고 있습니다. 우리나라의 빈약한 컴퓨터 교육으로 인하여 입학 전에는 컴퓨터 프로그래밍에 문외한이다시피 했던 학생들이 졸업할 때는 초급 프로그래밍 기술자가 되는 광경을 쉽게 찾아볼
   🧠 벡터 길이: 384
   🔢 벡터 앞 5개 값: [-0.06271791  0.07824273  0.06209158 -0.06104202 -0.00155998]
---------------------------------------------------------------------

In [30]:
# 완전 초기화 ------ 신중히 사용하기 바람 -------
"""
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

# 컬렉션 내의 모든 문서 ID를 가져와 삭제
existing_ids = collection.get(limit=None)['ids']
if existing_ids:
    collection.delete(ids=existing_ids)
    print(f"🗑️ 컬렉션 내 모든 데이터 삭제 완료: {len(existing_ids)}개 삭제됨")
else:
    print("✅ 삭제할 데이터가 없습니다. 컬렉션이 이미 비어있습니다.")
"""

'\nfrom chromadb import PersistentClient\n\nPERSIST_DIR = "/content/drive/MyDrive/chroma_index"\nCOLLECTION_NAME = "knu_collection"\n\nclient = PersistentClient(path=PERSIST_DIR)\ncollection = client.get_or_create_collection(name=COLLECTION_NAME)\n\n# 컬렉션 내의 모든 문서 ID를 가져와 삭제\nexisting_ids = collection.get(limit=None)[\'ids\']\nif existing_ids:\n    collection.delete(ids=existing_ids)\n    print(f"🗑️ 컬렉션 내 모든 데이터 삭제 완료: {len(existing_ids)}개 삭제됨")\nelse:\n    print("✅ 삭제할 데이터가 없습니다. 컬렉션이 이미 비어있습니다.")\n'